In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

In [4]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [5]:
# Grid parameters
N_S = 100   # Spatial points
N_tau = 100 # Temporal points

In [6]:
# ================== PINN Architecture ==================
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 20),  # Input: (S, tau)
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1)   # Output: V(S,tau)
        )
        
    def forward(self, S, tau):
        Stau = torch.cat([S, tau], dim=1)
        return self.net(Stau)

In [18]:
# ================== Loss PDE ==================
def pde_dynamic(pinn, coef=0.1):

    S = torch.rand(N_S, 1, requires_grad=True)
    tau = torch.rand(N_tau, 1, requires_grad=True)
    
    V = pinn(S, tau)
    V_tau = torch.autograd.grad(V.sum(), tau, create_graph=True)[0]
    V_S = torch.autograd.grad(V.sum(), S, create_graph=True)[0]
    V_SS = torch.autograd.grad(V_S.sum(), S, create_graph=True)[0]
    
    return torch.mean((V_tau - coef * V_SS) ** 2)


def boundary_condition(pinn):
    
    boundary_S0 = torch.zeros(N_tau, 1)
    boundary_S1 = torch.ones(N_tau, 1)
    boundary_tau = torch.rand(N_tau, 1, requires_grad=True)
    boundary_V0 = torch.zeros(N_tau, 1)
    boundary_V1 = torch.zeros(N_tau, 1)

    pred_boundary0 = pinn(boundary_S0, boundary_tau)
    pred_boundary1 = pinn(boundary_S1, boundary_tau)
    loss_boundary = torch.mean((pred_boundary0 - boundary_V0)**2) + \
                    torch.mean((pred_boundary1 - boundary_V1)**2)
    
    return loss_boundary

def initial_condition(pinn):
    # Initial condition (tau=0): V(S,0) = sin(πS)
    initial_S = torch.rand(N_S, 1, requires_grad=True)
    initial_tau = torch.zeros(N_S, 1)
    initial_V = torch.sin(np.pi * initial_S)

    pred_initial = pinn(initial_S, initial_tau)
    loss_initial = torch.mean((pred_initial - initial_V)**2)
    return loss_initial

In [19]:
def visualize(pinn):
    # ================== Visualization ==================
    # Generate test data
    S_test = torch.linspace(0, 1, N_S).view(-1, 1)
    tau_test = torch.linspace(0, 1, N_tau).view(-1, 1)
    S_grid, Tau_grid = torch.meshgrid(S_test.squeeze(), tau_test.squeeze())

    # Predict solution
    with torch.no_grad():
        V_pred = pinn(S_grid.reshape(-1, 1), Tau_grid.reshape(-1, 1)).reshape(N_S, N_tau).numpy()

    # Create interactive 3D plot with plotly
    fig = go.Figure(data=[go.Surface(
        x=S_grid.numpy(),
        y=Tau_grid.numpy(),
        z=V_pred,
        colorscale='jet',
        opacity=0.8,
    )])

    # Update layout
    fig.update_layout(
        title='Interactive 3D Heat Equation Solution',
        scene=dict(
            xaxis_title='Position (S)',
            yaxis_title='Time (tau)',
            zaxis_title='V(S,tau)',
        ),
        autosize=True,
        width=900,
        height=700,
    )

    # Show the plot
    fig.show()

In [24]:
def train_network(diffusion_coefficient = 0.1):
    # ================== Training Setup ==================
    pinn = PINN()
    optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)


    # ================== Training Loop with EMA Early Stopping ==================
    epochs = 5000
    train_losses = []

    # EMA parameters
    ema_loss = None
    alpha = 0.1       # Smoothing factor
    patience = 200     # Epochs to wait before stopping
    min_delta = 1e-5   # Minimum improvement threshold
    wait = 0           # Epochs since last improvement
    min_epochs = 100   # Minimum training epochs before checking

    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Compute losses

        
        loss_initial = initial_condition(pinn)
        loss_boundary = boundary_condition(pinn)
        loss_physics = pde_dynamic(pinn, coef = diffusion_coefficient)
        loss = loss_initial + loss_boundary + loss_physics

        train_losses.append(loss.item())
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # EMA-based early stopping
        ema_loss = alpha * loss.item() + (1-alpha)*ema_loss if ema_loss else loss.item()
        
        if epoch > min_epochs:
            if loss.item() < ema_loss - min_delta:
                wait = 0  # Reset counter if improving
            else:
                wait += 1
                if wait >= patience:
                    print(f"\nEarly stopping at epoch {epoch}")
                    print(f"Final loss: {loss.item():.6f} (EMA: {ema_loss:.6f})")
                    break
        
        if epoch % 500 == 0:
            print(f"Epoch {epoch:4d} | Loss: {loss.item():.6f} | EMA: {ema_loss:.6f}")
    
    return pinn

In [28]:
pinn = train_network(0.2)
visualize(pinn)

Epoch    0 | Loss: 0.522389 | EMA: 0.522389
Epoch  500 | Loss: 0.021394 | EMA: 0.023294
Epoch 1000 | Loss: 0.002450 | EMA: 0.002315
Epoch 1500 | Loss: 0.000724 | EMA: 0.000806
Epoch 2000 | Loss: 0.000363 | EMA: 0.000390
Epoch 2500 | Loss: 0.000281 | EMA: 0.000293
Epoch 3000 | Loss: 0.000235 | EMA: 0.000196
Epoch 3500 | Loss: 0.000256 | EMA: 0.000168
Epoch 4000 | Loss: 0.000120 | EMA: 0.000098
Epoch 4500 | Loss: 0.000151 | EMA: 0.000130
